# Creating a map with F1 information

## Circuits, Constructors and Drivers around the Globe

Installing the required modules

In [1]:
#pip install wikipedia
#pip install folium

In [168]:
import folium #module to create maps
import pandas as pd #data science module
import pyodbc #to connect to 

In [169]:
import wikipedia as wiki

Connecting python to the Azure db.

In [170]:
server = 'f1server.database.windows.net'
database = 'WackyRacesF1'
username = 'PDS'
password = '{Formula1}'   
driver= '{ODBC Driver 17 for SQL Server}'

Running SQL queries using Python to import the required information into Pandas dataframes.

In [171]:
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
constructors_query = "SELECT DISTINCT c.constructorId, constructorRef, c.name, nationality, c.url FROM constructors c LEFT JOIN constructorResults cr ON c.constructorId = cr.constructorId LEFT JOIN races r ON r.raceId = cr.raceId WHERE year >= 2015 and year <= 2022;"
constructors = pd.read_sql(constructors_query, cnxn)
circuits_query = "SELECT DISTINCT c.circuitId, c.name, c.location, c.country, lat, lng, alt, c.url FROM circuits c LEFT JOIN races r ON r.circuitId = c.circuitId WHERE year >= 2015 and year <= 2022;"
circuits_sql = pd.read_sql(circuits_query, cnxn)

Importing csvs with information on drivers and circuits into additional Pandas dataframes.

In [172]:
drivers = pd.read_excel('f1_drivers_2015_2021.xlsx')
circuits = pd.read_excel('f1_circuits_2015_2021.xlsx')

In [173]:
constructors

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
2,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
3,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso
4,6,ferrari,Ferrari,Italian,http://en.wikipedia.org/wiki/Scuderia_Ferrari
5,9,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing
6,10,force_india,Force India,Indian,http://en.wikipedia.org/wiki/Racing_Point_Forc...
7,15,sauber,Sauber,Swiss,http://en.wikipedia.org/wiki/Sauber
8,51,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...
9,117,aston_martin,Aston Martin,British,http://en.wikipedia.org/wiki/Aston_Martin_in_F...


In [174]:
circuits

,season,circuit_id,lat,long,country
0,2015,albert_park,-37.84970,144.96800,Australia
1,2015,sepang,2.76083,101.73800,Malaysia
2,2015,shanghai,31.33890,121.22000,China
3,2015,bahrain,26.03250,50.51060,Bahrain
4,2015,catalunya,41.57000,2.26111,Spain
...,...,...,...,...,...
114,2020,imola,44.34390,11.71670,Italy
115,2020,istanbul,40.95170,29.40500,Turkey
116,2020,bahrain,26.03250,50.51060,Bahrain
117,2020,bahrain,26.03250,50.51060,Bahrain


Preparing the dataframe "circuits" to store unique information only on circuits. 

In [175]:
circuits_u = circuits.groupby(['lat','long','country'])['circuit_id'].count().reset_index()

In [176]:
circuits_u

,lat,long,country,circuit_id
0,-37.84970,144.96800,Australia,5
1,-23.70360,-46.69970,Brazil,5
2,1.29140,103.86400,Singapore,5
3,2.76083,101.73800,Malaysia,3
4,19.40420,-99.09070,Mexico,5
5,24.46720,54.60310,UAE,6
6,26.03250,50.51060,Bahrain,7
7,30.13280,-97.64110,USA,5
8,31.33890,121.22000,China,5
9,34.84310,136.54100,Japan,5


In [187]:
results = []
for i, row in circuits_u.iterrows():
    print(row['lat'],row['long'])
    #result = wiki.geosearch(row['lat'],row['long'])
    #results.append(result)

-37.8497 144.968
-23.7036 -46.6997
1.2914 103.864
2.76083 101.738
19.4042 -99.0907
24.4672 54.6031
26.0325 50.5106
30.1328 -97.6411
31.3389 121.22
34.8431 136.541
37.227 -8.6267
40.3725 49.8533
40.9517 29.405
41.57 2.26111
43.2506 5.79167
43.4057 39.9578
43.7347 7.42056
43.9975 11.3719
44.3439 11.7167
45.5 -73.5228
45.6156 9.28111
47.2197 14.7647
47.5789 19.2486
49.3278 8.56583
50.3356 6.9475
50.4372 5.97139
52.0786 -1.01694


In [183]:
result

['1992 British Grand Prix',
 '2008 British Grand Prix',
 '1995 British Grand Prix',
 '2006 British Grand Prix',
 '2009 British Grand Prix',
 '2007 British Grand Prix',
 '2004 British Grand Prix',
 '2003 British Grand Prix',
 'Silverstone Circuit',
 '2005 British Grand Prix']

In [164]:
for i, coord in enumerate(drivers):
    print(coord)
    print(i)

DRIVER_ID
0
DRIVER
1
DRIVER_REF
2
DRIVER_CODE
3
DOB
4
NATIONALITY
5


In [136]:
circuits_sql

,circuitId,name,location,country,lat,lng,alt,url
0,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-378.497009,144.968002,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,Sepang International Circuit,Kuala Lumpur,Malaysia,276.083008,101.737999,18.0,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,Bahrain International Circuit,Sakhir,Bahrain,260.325012,505.105988,7.0,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.570000,226.110992,109.0,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,Istanbul Park,Istanbul,Turkey,409.516998,29.405001,130.0,http://en.wikipedia.org/wiki/Istanbul_Park
5,6,Circuit de Monaco,Monte-Carlo,Monaco,437.346985,742.056030,7.0,http://en.wikipedia.org/wiki/Circuit_de_Monaco
6,7,Circuit Gilles Villeneuve,Montreal,Canada,45.500000,-735.228027,13.0,http://en.wikipedia.org/wiki/Circuit_Gilles_Vi...
7,9,Silverstone Circuit,Silverstone,UK,520.786011,-101.694000,153.0,http://en.wikipedia.org/wiki/Silverstone_Circuit
8,10,Hockenheimring,Hockenheim,Germany,493.278015,856.583008,103.0,http://en.wikipedia.org/wiki/Hockenheimring
9,11,Hungaroring,Budapest,Hungary,475.789001,192.485992,264.0,http://en.wikipedia.org/wiki/Hungaroring


In [129]:
def get_wiki_info(df):
    summary = []

    for lat, lon in df:
        detail = wiki.geosearch(lat,lon)
        summary.append(detail)
        print(lat, lon)
        
    return summary

In [130]:
get_wiki_info(circuits['lat','lon'])

KeyError: ('lat', 'lon')

Complement circuits information with the information extracted from Azure SQL.

TypeError: string indices must be integers

In [ ]:
circuits = circuits.

In [53]:
drivers

,DRIVER_ID,DRIVER,DRIVER_REF,DRIVER_CODE,DOB,NATIONALITY
0,1,Lewis Hamilton,hamilton,HAM,1985-01-07,British
1,3,Nico Rosberg,rosberg,ROS,1985-06-27,German
2,4,Fernando Alonso,alonso,ALO,1981-07-29,Spanish
3,8,Kimi Raikkonen,raikkonen,RAI,1979-10-17,Finnish
4,9,Robert Kubica,kubica,KUB,1984-12-07,Polish
5,13,Felipe Massa,massa,MAS,1981-04-25,Brazilian
6,16,Adrian Sutil,sutil,SUT,1983-01-11,German
7,18,Jenson Button,button,BUT,1980-01-19,British
8,20,Sebastian Vettel,vettel,VET,1987-07-03,German
9,154,Romain Grosjean,grosjean,GRO,1986-04-17,French


In [34]:
def get_wiki_summary(df):
    summary = []

    for name in circuits['name']:
        detail = wiki.WikipediaPage(title=name).summary
        details.append(detail)
        
    return summary

In [36]:
circuits

,season,circuit_id,lat,long,country
0,2015,albert_park,-37.84970,144.96800,Australia
1,2015,sepang,2.76083,101.73800,Malaysia
2,2015,shanghai,31.33890,121.22000,China
3,2015,bahrain,26.03250,50.51060,Bahrain
4,2015,catalunya,41.57000,2.26111,Spain
...,...,...,...,...,...
114,2020,imola,44.34390,11.71670,Italy
115,2020,istanbul,40.95170,29.40500,Turkey
116,2020,bahrain,26.03250,50.51060,Bahrain
117,2020,bahrain,26.03250,50.51060,Bahrain


In [35]:
circuit = get_wiki_details(circuits['details'])

KeyError: 'details'

In [199]:
def generate_map(df):
    
    my_map = folium.Map(location=[38.73241897525329, -9.160338893258317],
                    zoom_start=10,
                    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                    attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
                   )
    
    for index, row in df.iterrows():
        
        html=f"""
        <h1><img src='data:image/svg+xml;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz4KPCEtLSBHZW5lcmF0b3I6IEFkb2JlIElsbHVzdHJhdG9yIDI1LjIuMywgU1ZHIEV4cG9ydCBQbHVnLUluIC4gU1ZHIFZlcnNpb246IDYuMDAgQnVpbGQgMCkgIC0tPgo8c3ZnIHZlcnNpb249IjEuMSIgaWQ9IkxheWVyXzEiIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyIgeG1sbnM6eGxpbms9Imh0dHA6Ly93d3cudzMub3JnLzE5OTkveGxpbmsiIHg9IjBweCIgeT0iMHB4IgoJIHZpZXdCb3g9IjAgMCAxNTQ2LjIgOTcwLjIiIHN0eWxlPSJlbmFibGUtYmFja2dyb3VuZDpuZXcgMCAwIDE1NDYuMiA5NzAuMjsiIHhtbDpzcGFjZT0icHJlc2VydmUiPgo8c3R5bGUgdHlwZT0idGV4dC9jc3MiPgoJLnN0MHtmaWxsOm5vbmU7c3Ryb2tlOiMzNzQxNTE7c3Ryb2tlLXdpZHRoOjcuODU0NTtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQoJLnN0MXtmaWxsOm5vbmU7c3Ryb2tlOiNFRjQ0NDQ7c3Ryb2tlLXdpZHRoOjEuOTYzNjtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQoJLnN0MntmaWxsOm5vbmU7c3Ryb2tlOiMzQjgyRjY7c3Ryb2tlLXdpZHRoOjEuOTYzNjtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQoJLnN0M3tmaWxsOm5vbmU7c3Ryb2tlOiNGQkJGMjQ7c3Ryb2tlLXdpZHRoOjEuOTYzNjtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQo8L3N0eWxlPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDAiIGQ9Ik01NTMuOCwyMjkuOGMzMy4xLDY2LjUsNjIuNiwxMzUuMiw5NC4zLDIwMi4zCgljMTAuNCwyMi4xLDIwLjMsNDUuNSwzMi40LDY2LjhjOC40LDE4LjYsMTguNCwzOSwzMy40LDUzYzEyLjcsMTUuMywzNi4yLDE5LjIsNTUsMTUuN2MxNy0zLjUsMzIuNC0xMi44LDQ3LjEtMjEuNgoJYzIyLjYtMTQuOCw0Ni40LTI4LjgsNjguNy00NC4yYzEwOS42LTc0LjMsMjguMy0xMzMsODMuNS0yMThjMjMuOS0zOC42LDY4LjktNjMsMTEzLjktNjUuOGMyMC44LTMuMSw0My0wLjcsNjMuOC0yLjkKCWMyNS4yLTEuOSw1MC40LTcuOCw3NC42LTE0LjdjNDAuNy0xMi45LDgwLjEtMzEuOCwxMTguOC00OS4xYzIxLjYtMTIuOCw1MC41LTE4LjIsNjEuOC00Mi4yYzExLjMtMjYuOC0wLjEtNTEuMSwzNy4zLTYwLjkKCWMxMy45LTMsMjcuNCwyLjYsNDAuMyw1LjljMTMuMSwzLjMsMjYuMyw0LjQsMzkuMywzLjljMTguMy00LjMsMjctMjUuMiwxMC44LTM5LjNjLTExLjUtOC42LTI2LjktOS45LTQwLjMtOS44CgljLTIzMiw3LjQtNDY1LjIsNC4zLTY5Ny4xLDcuOWMtMTQyLjYsMC43LTI4NSwxLjMtNDI3LjEsMWMtMTgsMC42LTMxLDE0LjctMjguNSwzMi40YzguMywxNi4xLDE2LjEsMzUuMiwzLjksNTEuMQoJYy01LjQsMTAuOC0xOC4zLDE2LjItMjcuNSwyMi42Yy0xNy42LDExLjctMzcuMywyMC42LTUzLDM0LjRjLTI1LjcsMTguNS00My43LDQ1LjItNjAuOSw3MC43Yy0yOC4yLDQyLjMtNDguOSw4OC43LTY2LjgsMTM1LjUKCUM3Ni44LDUwMy41LDMwLjQsNjQ3LjYsMTcuOCw3OTYuNGMtMy40LDM0LTguOSw2Ny4xLTguOCwxMDAuMWM1LjQsMzMuOSwyNS42LDMzLjksNTQsMzUuM2MyOC41LDguNywzMiwyOC45LDYzLjgsMjkuNAoJYzE5LjUtMC40LDM4LjctNy41LDU2LTEzLjhjMTkuMS02LjUsMzguNy0xOC4xLDQzLjItMzguM2M1LjUtMzIuNC0yMC4yLTY0LjctMzYuMy05MC4zYy0xNy4xLTI4LjUtMzQuMy01Ny4xLTUwLjEtODUuNAoJYy01MS4xLTg0LjQtMjYtMjA2LjIsNTktMjU3LjJjMjAuOC0xNi4xLDU1LjUtMzcuMiw3NC42LTkuOGM3LjYsMTIuNSw5LjMsMjcuMiwxMy43LDQwLjNjNS41LDIzLjksMjguMSw0MS44LDUyLjEsMzkuMgoJYzE5LjctMi4xLDM5LjMtOS42LDU2LTE4LjdjMzYuMy0xOC42LDMxLjEtNjUuMSwxNC43LTk1LjNjLTE4LjMtNDAuMi00Ni43LTc0LjktNjYuOC0xMTIuOWMtMTcuOC0yOC4yLTM3LjItNjUtMTkuNi05Ni4yCgljNS4yLTE1LjcsMTcuOC0yNy41LDI4LjUtMzguM2M5LjMtMTEsMTktMjIsMjguNS0zMS40YzI0LjUtMjguMiw1NS45LTcxLDk3LjItNTMuOWMyNy41LDEzLjksMzkuNCw0NS41LDUwLjEsNzEuNwoJQzUzNS4yLDE5MS41LDU0NS4xLDIxMC4yLDU1My44LDIyOS44eiIvPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDEiIGQ9Ik01NTMuOCwyMjkuOGMzNi44LDc0LjQsNjkuOSwxNTAuOCwxMDUuMSwyMjUuOAoJYzMzLjQsNjYuMiw1NywxNTAuMywxNDcuMyw5Ni4yYzUxLTM1LjEsMTIxLTYxLDEzOC40LTEyNi43YzYtMjcuNSwwLjMtNTUuOCwyLTgzLjVjNS45LTQ0LjcsMzQuMi04NC41LDczLjYtMTA2CgljNDMuMS0yNS40LDk1LTE1LjgsMTQyLjQtMjIuNmM1Ny4yLTcuNCwxMTEuMS0zMS42LDE2My01NiIvPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDIiIGQ9Ik0xMzI1LjUsMTU3LjJjMTMuOC01LjcsMjgtMTIuOSw0MS4yLTE5LjZjMTMuOC01LjcsMjguMS0xNC4zLDM0LjQtMjguNQoJYzctMTIuOSw0LTI5LjEsMTAuOC00Mi4yYzcuMS0xNCwyNi4xLTIxLjksNDEuMi0xOC43YzkuOS0wLjQsMTkuNyw1LjQsMjkuNSw2LjljMTMsMS45LDI3LjYsNS44LDQwLjMsMQoJYzI1LjgtMTEuNiwxNC41LTM5LjUtMTAuOC00NS4yYy0xNi40LTMuMy0zMy41LTEuNi01MC4xLTFjLTIxLjIsMC4yLTQyLjcsMS02My44LDJjLTMzOS45LDIuOC02ODAsNi4zLTEwMjAuMSw1LjkKCWMtMjYuNS0yLjktNTEsMTAuOC0zOS4zLDQwLjNjMjIuMiwzNC4yLTcuOSw1Ny0zNi4zLDcxLjdjLTM0LDE5LjQtNjYuOSw0My04OC40LDc2LjZjLTQyLjUsNTctNzAuMywxMjMtOTQuMywxODkuNQoJQzcwLjYsNTIxLjMsMzEuOCw2NTIsMTguNyw3ODYuNSIvPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDMiIGQ9Ik0xOC43LDc4Ni41Yy0yLjksMjMuNi00LjgsNDguMi03LjksNzEuN2MtMS41LDIyLTcuNiw0OS41LDExLjgsNjUuOAoJYzEwLDcuMSwyMi42LDcuMSwzNC40LDYuOWM3LjksMC44LDE1LjQsNSwyMi42LDcuOWMyNy42LDI1LjksNTAuNCwyNy40LDg1LjQsMTQuN2M0MC43LTEwLjUsNzYuOC0zMi44LDU2LTc5LjUKCWMtMTAuMy0yNi45LTI5LjItNDkuNy00My4yLTc0LjZjLTIwLjMtMzYuNC00Ni4zLTcxLjQtNTcuOS0xMTEuOWMtMTIuNS00NC40LTYuNS05My4xLDEwLjgtMTM1LjVjMjAuMi00MS45LDU3LjMtNzMuNSw5OC4yLTk0LjMKCWMyMi4yLTExLjgsNDcuMSwwLjQsNTEuMSwyNS41YzcuOCwzMCwxNiw2MS4yLDUzLDYyLjhjMTQuNiwwLjksMjguOS00LjgsNDIuMi05LjhjMjQtNi41LDQ2LjUtMjUuOSw0Ny4xLTUyCgljMS4yLTMzLjUtMTcuNS02My42LTM0LjQtOTEuM2MtMTcuNS0yOC4zLTM2LjEtNTYuMi01Mi04NS40Yy0xOS43LTMxLTI4LjUtNjYuNC01LjktOTguMmMxOC4yLTIzLDM5LjItNDQuMSw1OC45LTY1LjgKCWMxMi44LTE2LDI3LjYtMzEuNiw0NS4yLTQyLjJjMTkuNi05LjIsNDYuNC03LjcsNTkuOSwxMC44YzE2LjYsMTUuOCwyNSwzOC4xLDMzLjQsNTguOWM3LjYsMTkuOSwxNy44LDM5LjQsMjYuNSw1OC45Ii8+Cjwvc3ZnPgo='/></p>
        </h1>
        <h1><p> {row['name']}</p></h1>
        <p><p>Circuit information:</p>
        <ul>
            <li>Country name: {row['country']}</li>
            <li>City name: {row['location']}</li>
        <details>
            <summary>More details</summary>
            {row['details']}
        </details>     
        </ul>
        </p>
        <p>To know more: <a href={row['url']}>link</a></p>
        """
        
        iframe = folium.IFrame(html=html, width=300, height=300)
        
        popup = folium.Popup(iframe, max_width=2650)
        
        folium.Marker(location=[row['lat'],row['lng']],popup=popup,tooltip=row['name'],radius=3, color='Black').add_to(my_map)

        
    return my_map

In [201]:
generate_map(circuits_sql)

KeyError: 'details'

In [197]:
def get_coords(df):
    details=[]

    for name in circuits['name']:
        try:
            detail = wiki.WikipediaPage(title=name).coordinates
            lat, long = detail
            details.append((lat,long))
        except:
            if name == "Nürburgring":
                details.append(('1','2'))

In [198]:
details.info()

NameError: name 'details' is not defined

In [ ]:
lat, long = details[0]

NameError: name 'details' is not defined

In [35]:
type(lat)

decimal.Decimal

In [ ]:
print(lat,long)